# **Introduction**

This notebook is for performing hyperparameter sweeps on the algorithms that have been designed thus far.

# **Import Packages**

This section imports the necessary packages.

In [ ]:
# Import these packages:
import gymnasium as gym
import numpy as np
from tqdm import tqdm 
import matplotlib.pyplot as plt

import json
import itertools
from frozen_lake_classes import GLIE_MC_Agent, SARSA_0_Agent, SARSA_L_Agent